In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [2]:
data = pd.read_csv('AmesHousing.txt',sep = '\t')

#Separating rows into the train rows and the test rows
train = data[:1460].copy()
test = data[1460:].copy()
#the target column would be the column "SalePrice"
target = "SalePrice"

print(train.columns)

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
      

In [3]:
#Counting how many rows contain zero entries
train_null_counts = train.isnull().sum()
# print(train_null_counts)
#Series containing no zero rows
df_no_mv_series = train_null_counts[train_null_counts==0]
#data frame with the columns with no missing values
df_no_mv = train[df_no_mv_series.index]
print(df_no_mv.columns)

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Area', 'Street',
       'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope',
       'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Exter Qual', 'Exter Cond', 'Foundation', 'Heating',
       'Heating QC', 'Central Air', 'Electrical', '1st Flr SF', '2nd Flr SF',
       'Low Qual Fin SF', 'Gr Liv Area', 'Full Bath', 'Half Bath',
       'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd',
       'Functional', 'Fireplaces', 'Garage Cars', 'Garage Area', 'Paved Drive',
       'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch',
       'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold',
       'Sale Type', 'Sale Condition', 'SalePrice'],
      dtype='object')


In [4]:
print(train['Sale Condition'].value_counts())

Normal     1267
Abnorml      98
Partial      63
Family       18
Alloca       14
Name: Sale Condition, dtype: int64


In [5]:
#We are selecting the columns that have objects in them, and counting the different categories in each column
text_cols = df_no_mv.select_dtypes(include=['object']).columns
#Printing the number of categories for each column
for col in text_cols:
    print(col+":", len(train[col].unique()))

MS Zoning: 6
Street: 2
Lot Shape: 4
Land Contour: 4
Utilities: 3
Lot Config: 5
Land Slope: 3
Neighborhood: 26
Condition 1: 9
Condition 2: 6
Bldg Type: 5
House Style: 8
Roof Style: 6
Roof Matl: 5
Exterior 1st: 14
Exterior 2nd: 16
Exter Qual: 4
Exter Cond: 5
Foundation: 6
Heating: 6
Heating QC: 4
Central Air: 2
Electrical: 4
Kitchen Qual: 5
Functional: 7
Paved Drive: 3
Sale Type: 9
Sale Condition: 5


In [6]:
#Selecting the columns that are categorical
text_cols = df_no_mv.select_dtypes(include=['object']).columns
#Converting the values into categorical
for col in text_cols:
    train[col] = train[col].astype('category')
#Obtaining the same result as before, after the change into categorical    
for col in text_cols:
    print(col+":", len(train[col].unique()))

MS Zoning: 6
Street: 2
Lot Shape: 4
Land Contour: 4
Utilities: 3
Lot Config: 5
Land Slope: 3
Neighborhood: 26
Condition 1: 9
Condition 2: 6
Bldg Type: 5
House Style: 8
Roof Style: 6
Roof Matl: 5
Exterior 1st: 14
Exterior 2nd: 16
Exter Qual: 4
Exter Cond: 5
Foundation: 6
Heating: 6
Heating QC: 4
Central Air: 2
Electrical: 4
Kitchen Qual: 5
Functional: 7
Paved Drive: 3
Sale Type: 9
Sale Condition: 5


In [7]:
dummy_cols = pd.DataFrame()
for col in text_cols:
    col_dummies = pd.get_dummies(train[col])
    train = pd.concat([train, col_dummies], axis=1)
    del train[col]
print(train.columns)

Index(['Order', 'PID', 'MS SubClass', 'Lot Frontage', 'Lot Area', 'Alley',
       'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add',
       ...
       'ConLI', 'ConLw', 'New', 'Oth', 'WD ', 'Abnorml', 'Alloca', 'Family',
       'Normal', 'Partial'],
      dtype='object', length=236)


In [8]:
#In this case, 'Year Remod/Add' and 'Year Built' do not represent important information for a linear regression model
train['years_until_remod'] = train['Year Remod/Add'] - train['Year Built']

In [15]:
# data = pd.read_csv('AmesHousing.txt', delimiter="\t")
# train = data[0:1460]
# test = data[1460:]

#We are counting the number of columns that have less than 40% of missing rows 
train_null_counts = train.isnull().sum()
#The mask for selecting columns that have less than 40% of missing rows
mask = (train_null_counts>0) & (train_null_counts<584)
#Selecting the columns in the dataframe with less than 40% missing rows
df_missing_values = train[train_null_counts[mask].index]
#the list of missing rows for each column
print(df_missing_values.isnull().sum())
print("\n")
#the dtypes of each column
print(df_missing_values.dtypes)

Lot Frontage      249
Mas Vnr Type       11
Mas Vnr Area       11
Bsmt Qual          40
Bsmt Cond          40
Bsmt Exposure      41
BsmtFin Type 1     40
BsmtFin SF 1        1
BsmtFin Type 2     41
BsmtFin SF 2        1
Bsmt Unf SF         1
Total Bsmt SF       1
Bsmt Full Bath      1
Bsmt Half Bath      1
Garage Type        74
Garage Yr Blt      75
Garage Finish      75
Garage Qual        75
Garage Cond        75
dtype: int64


Lot Frontage      float64
Mas Vnr Type       object
Mas Vnr Area      float64
Bsmt Qual          object
Bsmt Cond          object
Bsmt Exposure      object
BsmtFin Type 1     object
BsmtFin SF 1      float64
BsmtFin Type 2     object
BsmtFin SF 2      float64
Bsmt Unf SF       float64
Total Bsmt SF     float64
Bsmt Full Bath    float64
Bsmt Half Bath    float64
Garage Type        object
Garage Yr Blt     float64
Garage Finish      object
Garage Qual        object
Garage Cond        object
dtype: object


In [17]:
#This is a dataframe with the columns with a few missing values
float_cols = df_missing_values.select_dtypes(include=['float'])
print(float_cols)

      Lot Frontage  Mas Vnr Area  BsmtFin SF 1  BsmtFin SF 2  Bsmt Unf SF  \
0            141.0         112.0         639.0           0.0        441.0   
1             80.0           0.0         468.0         144.0        270.0   
2             81.0         108.0         923.0           0.0        406.0   
3             93.0           0.0        1065.0           0.0       1045.0   
4             74.0           0.0         791.0           0.0        137.0   
...            ...           ...           ...           ...          ...   
1455           NaN           0.0         735.0           0.0        257.0   
1456           NaN         227.0           0.0           0.0       1257.0   
1457          73.0         320.0         668.0           0.0        336.0   
1458          75.0         202.0         252.0           0.0        901.0   
1459           NaN         396.0           0.0           0.0       1055.0   

      Total Bsmt SF  Bsmt Full Bath  Bsmt Half Bath  Garage Yr Blt  
0     

In [16]:
#Filling missing values with the mean of each column
float_cols = float_cols.fillna(float_cols.mean())
print(float_cols)

      Lot Frontage  Mas Vnr Area  BsmtFin SF 1  BsmtFin SF 2  Bsmt Unf SF  \
0       141.000000         112.0         639.0           0.0        441.0   
1        80.000000           0.0         468.0         144.0        270.0   
2        81.000000         108.0         923.0           0.0        406.0   
3        93.000000           0.0        1065.0           0.0       1045.0   
4        74.000000           0.0         791.0           0.0        137.0   
...            ...           ...           ...           ...          ...   
1455     68.928984           0.0         735.0           0.0        257.0   
1456     68.928984         227.0           0.0           0.0       1257.0   
1457     73.000000         320.0         668.0           0.0        336.0   
1458     75.000000         202.0         252.0           0.0        901.0   
1459     68.928984         396.0           0.0           0.0       1055.0   

      Total Bsmt SF  Bsmt Full Bath  Bsmt Half Bath  Garage Yr Blt  
0     